In [1]:
from flair.data import Sentence
from flair.nn import Classifier
import spacy

nlp = spacy.load("en_core_web_lg")

/Users/zhuoran/Projects/GraphTrace/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/zhuoran/Projects/GraphTrace/env/lib/python3.9/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/tensor/python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


testing with spacy for dependency parser

In [2]:
log_str = "2016-09-28 04:30:31, Info                  CBS    SQM: Failed to start upload with file pattern: C:\Windows\servicing\sqm\*_std.sqm, flags: 0x2 [HRESULT = 0x80004005 - E_FAIL]"
doc = nlp(log_str)
for token in doc:
    print(token, token.tag_, token.dep_)


2016 CD nummod
- SYM punct
09 CD prep
- SYM punct
28 CD prep
04:30:31 CD meta
, , punct
Info NNP nmod
                  _SP dep
CBS NNP nmod
    _SP dep
SQM NNS appos
: : punct
Failed VBN ROOT
to TO aux
start VB xcomp
upload VB dobj
with IN prep
file NN compound
pattern NN pobj
: : punct
C:\Windows\servicing\sqm\*_std.sqm NN appos
, , punct
flags NNS conj
: : punct
0x2 . punct
[ XX dep
HRESULT NNP dep
= NFP dep
0x80004005 NNS ROOT
- : punct
E_FAIL ADD appos
] -RRB- punct


testing with flair for linker prediction

In [2]:
sentence = Sentence("2016-09-28 04:30:31, Info                  CBS    SQM: Failed to start upload with file pattern: C:\Windows\servicing\sqm\*_std.sqm, flags: 0x2 [HRESULT = 0x80004005 - E_FAIL]")


In [3]:
# load the NER tagger
tagger = Classifier.load('linker')
# run NER over sentence
tagger.predict(sentence)
# iterate over predicted entities and print
for label in sentence.get_labels():
    print(label)

2024-01-26 13:22:05,976 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [4]:
# syntactic chunk testing
syn_tagger = Classifier.load("chunk")
syn_tagger.predict(sentence)
for chunk in sentence.get_labels():
  print(chunk)

2024-01-26 13:22:12,907 SequenceTagger predicts: Dictionary with 47 tags: O, S-NP, B-NP, E-NP, I-NP, S-VP, B-VP, E-VP, I-VP, S-PP, B-PP, E-PP, I-PP, S-ADVP, B-ADVP, E-ADVP, I-ADVP, S-SBAR, B-SBAR, E-SBAR, I-SBAR, S-ADJP, B-ADJP, E-ADJP, I-ADJP, S-PRT, B-PRT, E-PRT, I-PRT, S-CONJP, B-CONJP, E-CONJP, I-CONJP, S-INTJ, B-INTJ, E-INTJ, I-INTJ, S-LST, B-LST, E-LST, I-LST, S-UCP, B-UCP, E-UCP, I-UCP, <START>, <STOP>
Span[0:2]: "2016-09-28 04:30:31" → NP (0.8472)
Span[3:6]: "Info                  CBS    SQM" → NP (0.9595)
Span[7:11]: "Failed to start upload" → VP (0.8208)
Span[11:12]: "with" → PP (1.0)
Span[12:14]: "file pattern" → NP (0.9868)
Span[15:16]: "C" → NP (0.8323)
Span[17:20]: "Windows\servicing" → NP (0.9608)
Span[23:24]: "std.sqm" → NP (0.3615)
Span[25:26]: "flags" → NP (0.7034)
Span[27:28]: "0x2" → NP (0.9582)
Span[29:30]: "HRESULT" → NP (0.998)
Span[31:32]: "0x80004005" → NP (0.6902)
Span[33:34]: "E" → NP (0.4441)
Span[35:36]: "FAIL" → NP (0.4823)


In [5]:
# relation extraction
ner_tagger = Classifier.load("ner-fast")
ner_tagger.predict(sentence)

entities = sentence.get_labels("ner")
for entity in entities:
    print(entity)

extractor = Classifier.load("relations")
extractor.predict(sentence)

relations = sentence.get_labels("relation")
for relation in relations:
    print(relations)

for label in sentence.get_labels("relation"):
    print(label)


2024-01-26 14:07:05,076 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


In [9]:
print(sentence)

Sentence[37]: "2016-09-28 04:30:31, Info                  CBS    SQM: Failed to start upload with file pattern: C:\Windows\servicing\sqm\*_std.sqm, flags: 0x2 [HRESULT = 0x80004005 - E_FAIL]"


In [10]:
for token in sentence:
    print(token)

Token[0]: "2016-09-28"
Token[1]: "04:30:31"
Token[2]: ","
Token[3]: "Info"
Token[4]: "CBS"
Token[5]: "SQM"
Token[6]: ":"
Token[7]: "Failed"
Token[8]: "to"
Token[9]: "start"
Token[10]: "upload"
Token[11]: "with"
Token[12]: "file"
Token[13]: "pattern"
Token[14]: ":"
Token[15]: "C"
Token[16]: ":\"
Token[17]: "Windows"
Token[18]: "\"
Token[19]: "servicing"
Token[20]: "\"
Token[21]: "sqm"
Token[22]: "\*_"
Token[23]: "std.sqm"
Token[24]: ","
Token[25]: "flags"
Token[26]: ":"
Token[27]: "0x2"
Token[28]: "["
Token[29]: "HRESULT"
Token[30]: "="
Token[31]: "0x80004005"
Token[32]: "-"
Token[33]: "E"
Token[34]: "_"
Token[35]: "FAIL"
Token[36]: "]"
